# Path e librerie
## Prima di modificare avvertire gli altri!

## Install

In [1]:
#import drive dir
from google.colab import drive
drive_dir = 'drive' ; 
drive.mount('/content/drive')

Mounted at /content/drive


## Import

In [2]:
import tweepy
import numpy as np
from tweepy import Stream, OAuthHandler
from tqdm.notebook import tqdm
# from tqdm import tqdm !! per Jupyter from tqdm.notebook non funziona, runnare questo
import pandas as pd
pd.set_option('display.max_colwidth', None) # setta il num. di caratteri visibili per ogni cella della colonna
# es. 50 = primi 50 caratteri
# None = testo completo

import io
import pprint
import os
import glob
import warnings
warnings.filterwarnings('ignore')


## Path per Colab

In [3]:
##########################
### ROOT
##########################

prj_root_dir = f'/content/{drive_dir}/MyDrive/nucleare/'
os.makedirs(prj_root_dir, exist_ok=True)

##########################
### SCRAPING: file dello scraping di ATP
##########################

scrapingATP_root_dir = prj_root_dir + 'scrapingATP/'
os.makedirs(scrapingATP_root_dir, exist_ok=True)

##########################
### ETL: 
##########################

etl_root_dir = prj_root_dir + 'ETL/'
os.makedirs(etl_root_dir, exist_ok=True)

##########################
### DWH
##########################

dw_root_dir = prj_root_dir + 'DWH/'
os.makedirs(dw_root_dir, exist_ok=True)

##########################
### SENTIMENT
##########################

sentiment_root_dir = prj_root_dir + 'SENTIMENT/'
os.makedirs(sentiment_root_dir, exist_ok=True)

##########################
### WORDEMBEDDING
##########################

embedding_root_dir = prj_root_dir + 'EMBEDDING/'
os.makedirs(embedding_root_dir, exist_ok=True)

##########################
### OUTPUT
##########################

output_root_dir = prj_root_dir + 'OUTPUT/'
os.makedirs(output_root_dir, exist_ok=True)

##########################
### STAGING ANTONELLO M.
##########################

st_man_root_dir = prj_root_dir + 'STAGING_MANENTI/'
os.makedirs(st_man_root_dir, exist_ok=True)

##########################
### STAGING ANTONELLO S.
##########################

tone_root_dir = prj_root_dir + 'tone/'
os.makedirs(tone_root_dir, exist_ok=True)


##########################
### ML MODELS
##########################

models_root_dir = prj_root_dir + 'MODELS/'
os.makedirs(models_root_dir, exist_ok=True)

## Path per Jupyter
Solo quando si usa Jupyter runnare, il percorso dovrebbe essere lo stesso per tutti/e

In [ ]:
##########################
### ROOT
##########################

drive_dir = f'G:\\My Drive'
prj_root_dir = f'{drive_dir}\\nucleare\\'

##########################
### SCRAPING
##########################

scrapingATP_root_dir = prj_root_dir + 'scrapingATP\\'

##########################
### ETL
##########################

etl_root_dir = prj_root_dir + 'ETL\\'

##########################
### DWH
##########################

dw_root_dir = prj_root_dir + 'DWH\\'

##########################
### SENTIMENT
##########################

sentiment_root_dir = prj_root_dir + 'SENTIMENT\\'

##########################
### WORDEMBEDDING
##########################

embedding_root_dir = prj_root_dir + 'EMBEDDING\\'

##########################
### OUTPUT
##########################

output_root_dir = prj_root_dir + 'OUTPUT\\'

##########################
### STAGING ANTONELLO M.
##########################

st_man_root_dir = prj_root_dir + 'STAGING_MANENTI\\'

# 6. BoW



## 6.1 preprocessing our dataset: lematizzation, stopwords, ngrams

In [26]:
import nltk ### libreria di linguistica (Stanford)
from nltk.stem import WordNetLemmatizer #### individua la radice linguistica delle parole: non tronca le parole come un stamming
nltk.download('wordnet') ## tassonomia lingua inglese
nltk.download('stopwords')

import warnings
from tqdm import tqdm
from pprint import pprint
from gensim.models.phrases import Phrases
from nltk.corpus import stopwords
pd.set_option('display.max_colwidth', None)

connectors = stopwords.words('english')
stop = stopwords.words('english')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [27]:
file_path = output_root_dir +'twitter_text_cleaned_lang_topic_det.csv'
df = pd.read_csv(file_path, encoding='utf-8')
df['text_cleaned'] = df['text_nonl_hash_men_http_num_simb']
print(df.isna().sum())
print(df.info())
print(df.columns)

id                                      0
user                                    0
date                                    0
text                                    0
favorite_count                          0
hashtags                                0
location                            33784
retweet                                 0
retweet_count                           0
followers_count                         0
in_reply_to_status_id               73270
user_mentions                           0
date_norm                               0
time_zone                               0
text_nonl                               0
text_nonl_hash                          0
text_nonl_hash_men                      0
text_nonl_hash_men_http                 0
text_nonl_hash_men_http_num             0
text_nonl_hash_men_http_num_simb        0
Topic                                   0
text_lang                               0
text_lan_code                           0
lang_confidence                   

In [28]:
df = df[['id', 'user', 'date', 'text', 'text_cleaned','text_lan_code', 'lang_confidence', 'prediction_reliable', 'topic']]
print(df.isna().sum())
df.info()
df.head()

id                     0
user                   0
date                   0
text                   0
text_cleaned           0
text_lan_code          0
lang_confidence        0
prediction_reliable    0
topic                  0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84310 entries, 0 to 84309
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   84310 non-null  int64  
 1   user                 84310 non-null  object 
 2   date                 84310 non-null  object 
 3   text                 84310 non-null  object 
 4   text_cleaned         84310 non-null  object 
 5   text_lan_code        84310 non-null  object 
 6   lang_confidence      84310 non-null  float64
 7   prediction_reliable  84310 non-null  bool   
 8   topic                84310 non-null  object 
dtypes: bool(1), float64(1), int64(1), object(6)
memory usage: 5.2+ MB


,id,user,date,text,text_cleaned,text_lan_code,lang_confidence,prediction_reliable,topic
0,1460367104297820165,mandt610,Mon Nov 15 22:00:24 +0000 2021,Rublev’s revenge 😤\n\n@AndreyRublev97 reverses the result from the 2020 #NittoATPFinals and gets the win over Tsitsipas! https://t.co/vQoi9wxPch,Rublev s revenge reverses the result from the and gets the win over Tsitsipas,en,98.0,True,['Rublev' 'Tsitsipas']
1,1460367139299282955,LeeFergusson,Mon Nov 15 22:00:32 +0000 2021,Red-hot start 🔥\n\n@AndreyRublev97 powers past 2019 champion Tsitsipas 6-4 6-4 to get his first win of the 2021 #NittoATPFinals https://t.co/mv9gTC9m0N,Red hot start powers past champion Tsitsipas to get his first win of the,en,98.0,True,['Rublev' 'Tsitsipas']
2,1460367219548823556,73Satta,Mon Nov 15 22:00:52 +0000 2021,Alexander Zverev #NittoATPFinals https://t.co/NN9iNnheHA,Alexander Zverev,sk,94.0,False,['Zverev']
3,1460367236573638665,bikson82,Mon Nov 15 22:00:56 +0000 2021,When the 🐺 met the 🦁\n\n@DjokerNole 🤝 @Ibra_official \n\n#NittoATPFinals https://t.co/za2aB5HnIJ,When the met the,en,94.0,True,['Djokovic']
4,1460367240960917507,BarthelSamantha,Mon Nov 15 22:00:57 +0000 2021,💙🤍💙🤍🇬🇷 #Tsitsipas #NittoATPFinals https://t.co/8gFUvYyqiU,,un,0.0,False,['Tsitsipas']


In [29]:
print(f'original len: {len(df)}')

dff = df[df['text_lan_code'].isin(['en'])]
eng_len = len(dff)
print(f'english len: {eng_len}')

dff = dff[dff['prediction_reliable'] == True]
dff = dff[dff['lang_confidence'] > 96]

print(f'filtered len: {len(dff)}')
print(f'loss: {eng_len - len(dff)}')

print(dff.isna().sum())
print(dff.groupby(['text_lan_code', 'prediction_reliable']).size())
print(dff.groupby(['text_lan_code', 'lang_confidence']).size())

original len: 84310
english len: 49252
filtered len: 36557
loss: 12695
id                     0
user                   0
date                   0
text                   0
text_cleaned           0
text_lan_code          0
lang_confidence        0
prediction_reliable    0
topic                  0
dtype: int64
text_lan_code  prediction_reliable
en             True                   36557
dtype: int64
text_lan_code  lang_confidence
en             97.0                5329
               98.0               16647
               99.0               14581
dtype: int64


In [30]:
df = dff.copy()

lemmatizer = WordNetLemmatizer()
df['text_cleaned'] = df['text_cleaned'].str.lower().str.replace('[^\w\s]',' ', regex=True) #type(text) series of string
df['text_cleaned'] = df['text_cleaned'].str.lower().str.replace('\s+', ' ', regex=True) ## rimuovi spazi multipli

raw_text = df[~df['text_cleaned'].isna()]['text_cleaned']

print(len(raw_text))

####pulizia: sostituisco tutti i caratteri speciali con uno spazio
text = raw_text.str.lower()
print(len(text))
print(text.head())

###TOKENIZATION
df['token'] = text.str.split() ##text: series of list, text[0] list of string, text[0][0] string 

###LEMMATIZATION
df['ltoken'] = df.token.apply(lambda x: [lemmatizer.lemmatize(sent) for sent in x]) #text: series of list, text[0] list of string (lemmatized), text[0][0] string

# print(f'text[0][0]-({type(df.ltoken[0][0])})')
# print(f'text[0]-({type(df.ltoken[0])})')
# print(f'text-({type(df.ltoken)})')

print(f'text_cleaned: {df.text_cleaned[0]}')
print(f'token       : {df.token[0]}')
print(f'ltoken      : {df.ltoken[0]}')

# stop.extend(['good', 'bad', 'dont', 'many', 'love', 'excellent', 'would', 'perfect', 'even', 'great','nice', 'amazing'])


36557
36557
0                 rublev s revenge reverses the result from the and gets the win over tsitsipas 
1                      red hot start powers past champion tsitsipas to get his first win of the 
5                      red hot start powers past champion tsitsipas to get his first win of the 
6                 rublev s revenge reverses the result from the and gets the win over tsitsipas 
7    milan turin three years after their next gen semi final meeting amp battled again in italy 
Name: text_cleaned, dtype: object
text_cleaned: rublev s revenge reverses the result from the and gets the win over tsitsipas 
token       : ['rublev', 's', 'revenge', 'reverses', 'the', 'result', 'from', 'the', 'and', 'gets', 'the', 'win', 'over', 'tsitsipas']
ltoken      : ['rublev', 's', 'revenge', 'revers', 'the', 'result', 'from', 'the', 'and', 'get', 'the', 'win', 'over', 'tsitsipas']


In [31]:
number = 1
print(f'text_cleaned: {df.text_cleaned.to_list()[number]}')
print(f'token       : {df.token.to_list()[number]}')
print(f'ltoken      : {df.ltoken.to_list()[number]}')

text_cleaned: red hot start powers past champion tsitsipas to get his first win of the 
token       : ['red', 'hot', 'start', 'powers', 'past', 'champion', 'tsitsipas', 'to', 'get', 'his', 'first', 'win', 'of', 'the']
ltoken      : ['red', 'hot', 'start', 'power', 'past', 'champion', 'tsitsipas', 'to', 'get', 'his', 'first', 'win', 'of', 'the']


In [32]:
print(df.isna().sum())
len_target = 1
field_name ='token' 
counter = df[df[field_name].str.len() < len_target ][field_name].count()
print(f'{field_name} with len < {len_target}: {counter}')

id                     0
user                   0
date                   0
text                   0
text_cleaned           0
text_lan_code          0
lang_confidence        0
prediction_reliable    0
topic                  0
token                  0
ltoken                 0
dtype: int64
token with len < 1: 0


In [33]:
my_stop_words = []
stop.extend(my_stop_words)

df['ltoken'] = df.ltoken.apply(lambda row: [item for item in row if item not in stop])

df['stopwords'] = df.ltoken.apply(lambda row: [item for item in row if item in stop])
df['nostopwords'] = df.ltoken.apply(lambda row: [item for item in row if item not in stop])

bigram  = Phrases(df.ltoken, min_count=5, threshold=0.2, common_terms=connectors) #gensim 3, ad esempio "and" 
df['bigrams_afternostop'] = df.ltoken.apply(lambda row: bigram[row])

ngram   = Phrases(df.bigrams_afternostop, min_count=5, threshold=0.2, common_terms=connectors) # gensim 3
df['ngrams_afternostop'] = df.bigrams_afternostop.apply(lambda row: ngram[row])

df.bigrams_afternostop = df.bigrams_afternostop.apply(lambda bigr: [item for item in bigr if item not in stop])
df.ngrams_afternostop = df.ngrams_afternostop.apply(lambda ngr: [item for item in ngr if item not in stop])

bigram  = Phrases(df.nostopwords, min_count=5, threshold=0.2, common_terms=connectors) #gensim 3, ad esempio "and" 
df['bigrams_beforenostop'] = df.nostopwords.apply(lambda row: bigram[row])

ngram   = Phrases(df.bigrams_beforenostop, min_count=5, threshold=0.2, common_terms=connectors) # gensim 3
df['ngrams_beforenostop'] = df.bigrams_beforenostop.apply(lambda row: ngram[row])

df['bigrams_afternostop_sent'] = df.bigrams_afternostop.apply(lambda row: ' '.join(row))
df['bigrams_beforenostop_sent'] = df.bigrams_beforenostop.apply(lambda row: ' '.join(row))
df['ngrams_afternostop_sent'] = df.ngrams_afternostop.apply(lambda row: ' '.join(row))
df['ngrams_beforenostop_sent'] = df.ngrams_beforenostop.apply(lambda row: ' '.join(row))

In [34]:
# print(df.isna().sum())
len_target = 1
# 'bigrams_afternostop_sent' 
# 'bigrams_beforenostop_sent'
# 'ngrams_afternostop_sent'  
# 'ngrams_beforenostop_sent' 
field_name ='ngrams_beforenostop_sent' 
counter = df[df[field_name].str.len() < len_target ][field_name].count()
print(f'{field_name} with len < {len_target}: {counter}')

n = 10
print(f'df.bigrams_afternostop_sent[{n}] : {df.bigrams_afternostop_sent.to_list()[n]}')
print(f'df.bigrams_beforenostop_sent[{n}]: {df.bigrams_beforenostop_sent.to_list()[n]}')
print(f'df.ngrams_afternostop_sent[{n}]  : {df.ngrams_afternostop_sent.to_list()[n]}')
print(f'df.ngrams_beforenostop_sent[{n}] : {df.ngrams_beforenostop_sent.to_list()[n]}')

ngrams_beforenostop_sent with len < 1: 0
df.bigrams_afternostop_sent[10] : bravissimo_wa awarded_year end_atp tour_trophy presented_following win_casper ruud
df.bigrams_beforenostop_sent[10]: bravissimo_wa awarded_year end_atp tour_trophy presented_following win_casper ruud
df.ngrams_afternostop_sent[10]  : bravissimo_wa_awarded_year end_atp_tour_trophy presented_following_win_casper ruud
df.ngrams_beforenostop_sent[10] : bravissimo_wa_awarded_year end_atp_tour_trophy presented_following_win_casper ruud


In [35]:
df.columns

Index(['id', 'user', 'date', 'text', 'text_cleaned', 'text_lan_code',
       'lang_confidence', 'prediction_reliable', 'topic', 'token', 'ltoken',
       'stopwords', 'nostopwords', 'bigrams_afternostop', 'ngrams_afternostop',
       'bigrams_beforenostop', 'ngrams_beforenostop',
       'bigrams_afternostop_sent', 'bigrams_beforenostop_sent',
       'ngrams_afternostop_sent', 'ngrams_beforenostop_sent'],
      dtype='object')

### 6.1.1 Saving

In [36]:
df_saving = df.copy() #.replace('\n','', regex=True)
df_saving = df_saving[['id', 'user', 'date', 'text', 'text_cleaned','text_lan_code', 'lang_confidence', 'prediction_reliable', 'topic',
       'bigrams_afternostop_sent', 'bigrams_beforenostop_sent',
       'ngrams_afternostop_sent', 'ngrams_beforenostop_sent']]
saving_file_name = f'{sentiment_root_dir}ngrams_ourdataset_dataframe.csv'
df_saving.to_csv(saving_file_name, index=False, encoding='utf-8', sep =',' )
df_saving.isna().sum()

id                           0
user                         0
date                         0
text                         0
text_cleaned                 0
text_lan_code                0
lang_confidence              0
prediction_reliable          0
topic                        0
bigrams_afternostop_sent     0
bigrams_beforenostop_sent    0
ngrams_afternostop_sent      0
ngrams_beforenostop_sent     0
dtype: int64

## 6.2 Preprocesing Trustpilot dataset

In [41]:
pd.set_option('display.max_colwidth', None)
file_path = tone_root_dir +'trustpilot_review_for_ML.csv'
df_open = pd.read_csv(file_path)
len(df_open)

1652277

In [42]:
dftp = df_open.copy()

dftp = dftp[dftp.rating != 'rating']
dftp = dftp[~dftp.rating.isin([3,'3', 'rating'])]
dftp = dftp.drop_duplicates()
dftp.rating = dftp.rating.astype('int') 
print(len(dftp))
print(dftp.groupby(['rating']).size().index)
print(dftp.groupby(['rating']).size())

df1 = dftp[dftp.rating == 1 ]
df2 = dftp[dftp.rating == 2 ]
df4 = dftp[dftp.rating == 4 ]
df5 = dftp[dftp.rating == 5 ]

df4 = df4.sample(len(df2), random_state= 123)
df5 = df5.sample(len(df1), random_state= 123)
print(f'df1: {len(df1)}')
print(f'df2: {len(df2)}')
print(f'df4: {len(df4)}')
print(f'df5: {len(df5)}')

dftp = df1.copy()
dftp = dftp.append(df2)
dftp = dftp.append(df4)
dftp = dftp.append(df5)
print(dftp.groupby(['rating']).size())
print(dftp.groupby(['rating']).size())
print(len(dftp))

dftp.to_csv(tone_root_dir +'balanced_trustpilot_review_for_ML.csv', index=False, encoding='utf-8')

1012482
Int64Index([1, 2, 4, 5], dtype='int64', name='rating')
rating
1     88110
2     23884
4     86730
5    813758
dtype: int64
df1: 88110
df2: 23884
df4: 23884
df5: 88110
rating
1    88110
2    23884
4    23884
5    88110
dtype: int64
rating
1    88110
2    23884
4    23884
5    88110
dtype: int64
223988


In [43]:
df_open = pd.read_csv(tone_root_dir +'balanced_trustpilot_review_for_ML.csv', encoding='utf-8')
print(df_open.groupby(['rating']).size())
print(len(df_open))
df_open.rating.value_counts()

rating
1    88110
2    23884
4    23884
5    88110
dtype: int64
223988


5    88110
1    88110
4    23884
2    23884
Name: rating, dtype: int64

In [45]:
df = df_open.copy()
print(df.isna().sum())
df.rating.value_counts()

def binary(row):
    if row['rating'] > 3:
        val = 1
    else:
        val = 0
    return val

df['y'] = df.apply(binary, axis=1) ### axis = 1 --> sulle righe
print(len(df))
#escludo dal dataset le review null
df = df[~(df['comment'].isna())]
print(len(df))
#recupero dal dataset i testi delle reviews (X)
raw_text = df['comment']

print(raw_text[1])
#recupero la lista dei binary weight (y)
y = df['y'].tolist()


comment    0
rating     0
dtype: int64
223988
223988
This product is awesome! Super helpful with the nerve damage sustained from a dog bit incident. Couldnt recommended a better product


In [50]:
import nltk ### libreria di linguistica (Stanford)
nltk.download('wordnet') ## tassonomia lingua inglese

from tqdm import tqdm
from nltk.stem import WordNetLemmatizer #### individua la radice linguistica delle parole: non tronca le parole come un stamming
lemmatizer = WordNetLemmatizer()

raw_text = df[~df['comment'].isna()].loc[:, 'comment']
print(len(raw_text))

####pulizia: sostituisco tutti i caratteri speciali con uno spazio
text = raw_text.str.lower().str.replace('[^\w\s]',' ', regex=True) #type(text) series of string
text = text.str.lower().str.replace('\s+', ' ', regex=True) ## rimuovi spazi multipli
# text = raw_text.str.replace('!"%&()?^,.-;:_', ' ', regex=False) ## rimuovi spazi multipli
print(len(text))

# ##TOKENIZATION
df['token'] = text.str.split() ##text: series of list, text[0] list of string, text[0][0] string

# ###LEMMATIZATION
df['ltoken'] = df.token.apply(lambda x: [lemmatizer.lemmatize(sent) for sent in x]) #text: series of list, text[0] list of string (lemmatized), text[0][0] string

print(f'text[0][0]-({type(df.token[3][0])})')
print(f'text[0]-({type(df.token[3])})')
print(f'text-({type(df.token)})')
print(f'text[0][0]-({(df.token[3][0])})')
print(f'text[0]-({(df.token[3])})')
# print(f'text-({(df.token)})')
print(df.isna().sum())


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
223988
223988
text[0][0]-(<class 'str'>)
text[0]-(<class 'list'>)
text-(<class 'pandas.core.series.Series'>)
text[0][0]-(i)
text[0]-(['i', 'am', 'pretty', 'frustrated', 'with', 'him', 'my', 'boyfriend', 'decided', 'to', 'buy', 'me', 'one', 'of', 'the', 'buffalo', 'bags', 'he', 'ordered', 'it', 'back', 'in', 'october', 'and', 'the', 'at', 'the', 'time', 'the', 'website', 'stated', '3', '4', 'to', 'weeks', 'and', 'it', 'would', 'be', 'sent', 'out', 'he', 'also', 'accepted', 'the', 'money', 'we', 'have', 'tried', 'and', 'tried', 'to', 'talk', 'to', 'him', 'and', 'he', 'ignores', 'messages', 'he', 'messaged', 'back', 'once', '11', 'weeks', 'later', 'and', 'he', 'is', 'still', 'ignoring', 'us', 'very', 'very', 'disappointed', 'i', 'would', 'love', 'to', 'support', 'his', 'work', 'as', 'it', 'a', 'gorgeous', 'but', 'he', 'is', 'sure', 'it', 'making', 'it', 'hard', 'he', 'could',

In [51]:
stop.extend(['good', 'bad', 'dont', 'many', 'love', 'excellent', 'would', 'perfect', 'even', 'great','nice', 'amazing'])
my_stop_words = []

stop.extend(my_stop_words)

df['ltoken'] = df.ltoken.apply(lambda row: [item for item in row if item not in stop])

# df['stopwords'] = df.ltoken.apply(lambda row: [item for item in row if item in stop])
# df['nostopwords'] = df.ltoken.apply(lambda row: [item for item in row if item not in stop])



In [52]:
df['ltoken']

0                                                                                                                                                                                                                                                                                                                                                                                                                 [tttsfddfhfdhdhdhdhf]
1                                                                                                                                                                                                                                                                                                               [product, awesome, super, helpful, nerve, damage, sustained, dog, bit, incident, couldnt, recommended, better, product]
2                                                                                                                                                       

In [53]:
number = 3
print(f'comment     : {df.comment.to_list()[number]}')
print(f'token       : {df.token.to_list()[number]}')
print(f'ltoken      : {df.ltoken.to_list()[number]}')

print(df.isna().sum())

len_target = 1
field_name ='ltoken' 

counter = df[df[field_name].str.len() < len_target ][field_name].count()
print(f'{field_name} with len < {len_target}: {counter}/{len(df)}')
df = df[df['ltoken'].str.len() > 0]

counter = df[df[field_name].str.len() < len_target ][field_name].count()
print(f'{field_name} with len < {len_target}: {counter}/{len(df)}')
df.isna().sum()

comment     : I am pretty frustrated with him! My boyfriend decided to buy me one of the buffalo bags! He ordered it back in October and the at the time the website stated 3-4 to weeks and it would be sent out! He also accepted the money! We have tried and tried to talk to him and he ignores messages, he messaged back once! 11 weeks later and he is still ignoring us! Very very disappointed! I would love to support his work as it a gorgeous but he is sure it making it hard! He could at least message back... saying anything, possibly a hey I am super busy and I would be happy and understanding. Very disappointed!!!
token       : ['i', 'am', 'pretty', 'frustrated', 'with', 'him', 'my', 'boyfriend', 'decided', 'to', 'buy', 'me', 'one', 'of', 'the', 'buffalo', 'bags', 'he', 'ordered', 'it', 'back', 'in', 'october', 'and', 'the', 'at', 'the', 'time', 'the', 'website', 'stated', '3', '4', 'to', 'weeks', 'and', 'it', 'would', 'be', 'sent', 'out', 'he', 'also', 'accepted', 'the', 'money', 'we',

comment    0
rating     0
y          0
token      0
ltoken     0
dtype: int64

In [54]:

bigram  = Phrases(df.ltoken, min_count=5, threshold=0.2, common_terms=connectors) #gensim 3, ad esempio "and" 
df['bigrams_afternostop'] = df.ltoken.apply(lambda row: bigram[row])

ngram   = Phrases(df.bigrams_afternostop, min_count=5, threshold=0.2, common_terms=connectors) # gensim 3
df['ngrams_afternostop'] = df.bigrams_afternostop.apply(lambda row: ngram[row])

# df.bigrams_afternostop = df.bigrams_afternostop.apply(lambda bigr: [item for item in bigr if item not in stop])
df.ngrams_afternostop = df.ngrams_afternostop.apply(lambda ngr: [item for item in ngr if item not in stop])

bigram  = Phrases(df.nostopwords, min_count=20, threshold=0.2, common_terms=connectors) #gensim 3, ad esempio "and" 
df['bigrams_beforenostop'] = df.nostopwords.apply(lambda row: bigram[row])

ngram   = Phrases(df.bigrams_beforenostop, min_count=20, threshold=0.2, common_terms=connectors) # gensim 3
df['ngrams_beforenostop'] = df.bigrams_beforenostop.apply(lambda row: ngram[row])


In [56]:

df['bigrams_afternostop_sent'] = df.bigrams_afternostop.apply(lambda row: ' '.join(row))
df['ngrams_afternostop_sent'] = df.ngrams_afternostop.apply(lambda row: ' '.join(row))

# df['bigrams_beforenostop_sent'] = df.bigrams_beforenostop.apply(lambda row: ' '.join(row))
# df['ngrams_beforenostop_sent'] = df.ngrams_beforenostop.apply(lambda row: ' '.join(row))

In [57]:

# df.bigrams_afternostop_sent = df.bigrams_afternostop_sent.str.replace('[^\w\s]',' ').str.replace('\s+', ' ', regex=True).str.strip()
# df.bigrams_beforenostop_sent = df.bigrams_beforenostop_sent.str.replace('[^\w\s]',' ').str.replace('\s+', ' ', regex=True).str.strip()
# df.ngrams_afternostop_sent = df.ngrams_afternostop_sent.str.replace('[^\w\s]',' ').str.replace('\s+', ' ', regex=True).str.strip()
# df.ngrams_beforenostop_sent = df.ngrams_beforenostop_sent.str.replace('[^\w\s]',' ').str.replace('\s+', ' ', regex=True).str.strip()

print(len(df))
df = df[~(df.bigrams_afternostop_sent.str.len() == 0)]
# df = df[~(df.bigrams_beforenostop_sent.str.len() == 0)]
df = df[~(df.ngrams_afternostop_sent.str.len() == 0)]
# df = df[~(df.ngrams_beforenostop_sent.str.len() == 0)]
print(len(df))

223815
223815


In [59]:


# print(df.isna().sum())
len_target = 1
for field_name in ['token', 'ltoken'
              ,'bigrams_afternostop_sent' 
              # ,'bigrams_beforenostop_sent'
              ,'ngrams_afternostop_sent'  
              # ,'ngrams_beforenostop_sent'
              ]: 
  counter = df[df[field_name].str.len() < len_target][field_name].count()
  print(f'{field_name} with len < {len_target}: {counter}')

n = 10
print(f'df.bigrams_afternostop_sent[{n}] : {df.bigrams_afternostop_sent.to_list()[n]}')
# print(f'df.bigrams_beforenostop_sent[{n}]: {df.bigrams_beforenostop_sent.to_list()[n]}')
print(f'df.ngrams_afternostop_sent[{n}]  : {df.ngrams_afternostop_sent.to_list()[n]}')
# print(f'df.ngrams_beforenostop_sent[{n}] : {df.ngrams_beforenostop_sent.to_list()[n]}')

df.bigrams_afternostop_sent[10] : ordered laser toy summer_paw april battery already_dead gave_feedback product_received response_never ordering establishment snarky response_feedback tried shaking laser toy still_doe work_said submit_complaint also untrue send_feedback summer_paw product_2 week_received response definitely_never ordering people_sell defective_product rude_arrogant
df.ngrams_afternostop_sent[10]  : ordered laser toy summer_paw april battery already_dead gave_feedback product_received response_never ordering establishment snarky response_feedback tried shaking laser toy still_doe work_said submit_complaint also untrue send_feedback summer_paw product_2 week_received response definitely_never ordering people_sell defective_product rude_arrogant


In [61]:
df_saving = df.copy()
df_saving = df_saving.loc[:, ['comment', 'rating', 
                      'bigrams_afternostop_sent',
                      # 'bigrams_beforenostop_sent', 
                      'ngrams_afternostop_sent',
                      # 'ngrams_beforenostop_sent',
                      'y']]
saving_file_name = f'{tone_root_dir}trustpilot_ready4_bow.csv'
df_saving.to_csv(saving_file_name, index=False, encoding='utf-8', sep =',' )
df_saving.isna().sum()

comment                     0
rating                      0
bigrams_afternostop_sent    0
ngrams_afternostop_sent     0
y                           0
dtype: int64

## 6.3 BOW

### 6.3.1 Fit tennis dictionary and save it

In [67]:
opening_file_name = f'{sentiment_root_dir}ngrams_ourdataset_dataframe.csv'
df_open_tennis = pd.read_csv(opening_file_name, encoding='utf-8', sep =',' )
df_open_tennis.isna().sum()

id                           0
user                         0
date                         0
text                         0
text_cleaned                 0
text_lan_code                0
lang_confidence              0
prediction_reliable          0
topic                        0
bigrams_afternostop_sent     0
bigrams_beforenostop_sent    0
ngrams_afternostop_sent      0
ngrams_beforenostop_sent     0
dtype: int64

In [68]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
import pickle

df_ten = df_open_tennis.copy()

vectorizer = CountVectorizer(ngram_range=(1, 1), max_features=1000)
# vectorizer = TfidfVectorizer(ngram_range=(1, 3), max_features=1000)
X = vectorizer.fit(df_ten['ngrams_afternostop_sent'].to_list())
feature_names = vectorizer.get_feature_names()
# print(feature_names)

model_filename = f"{models_root_dir}our_bow_dict_model.pkl"
with open(model_filename, 'wb') as file:
    pickle.dump(vectorizer, file)


In [ ]:
print(X)
print(feature_names[0:100])

CountVectorizer(max_features=1000)
['able', 'absolutely', 'absolutely_pathetic_schedule_tournament', 'account', 'ace', 'action', 'actually', 'add', 'age_player_atp_final', 'agree', 'ahead', 'ahead_debut_evening_ha', 'al', 'alert_new_high_roller', 'alex_zverev', 'alexander', 'alexander_second_defeat_medvedev', 'alexander_zverev', 'allez_amp_champion_second', 'almost', 'already', 'also', 'alternate', 'always', 'always_applauding_opponent_hit', 'amazing', 'amp', 'amp_defeat_ram_salisbury', 'amp_grande_ibra_uvek', 'amp_stay_unblemished', 'amp_two_time_champion', 'ana', 'andre_agassi_alexander_zverev', 'andrei_rublev', 'andrey', 'andrey_rublev', 'andrey_rublev_beat_stefanos', 'anime_murder_ok_john', 'another', 'answer', 'anyone', 'anyone_offends_know_far', 'anything', 'anyway', 'apart_djokovic_djokovic_least', 'around', 'ask', 'asked', 'atp', 'atp_final', 'atp_final_defending_champ', 'atp_final_sunday_pm', 'atp_tour', 'atp_tour_season', 'atp_world_tour_final', 'attack', 'australian_open_rol

### 6.3.2 TRUSTPILOT


In [69]:
df_open_pilot = pd.read_csv(f'{tone_root_dir}trustpilot_ready4_bow.csv', encoding='utf-8', sep =',', header=0 )

# df.bigrams_afternostop_sent = df.bigrams_afternostop_sent.str.replace('[^\w\s]',' ').str.replace('\s+', ' ', regex=True).str.strip()
# df.bigrams_beforenostop_sent = df.bigrams_beforenostop_sent.str.replace('[^\w\s]',' ').str.replace('\s+', ' ', regex=True).str.strip()
# df.ngrams_afternostop_sent = df.ngrams_afternostop_sent.str.replace('[^\w\s]',' ').str.replace('\s+', ' ', regex=True).str.strip()
# df.ngrams_beforenostop_sent = df.ngrams_beforenostop_sent.str.replace('[^\w\s]',' ').str.replace('\s+', ' ', regex=True).str.strip()

# print(len(df_open))
# df_open = df_open[~(df_open.bigrams_afternostop_sent.str.len() == 0)]
# df_open = df_open[~(df_open.bigrams_beforenostop_sent.str.len() == 0)]
# df_open = df_open[~(df_open.ngrams_afternostop_sent.str.len() == 0)]
# df_open = df_open[~(df_open.ngrams_beforenostop_sent.str.len() == 0)]
# print(len(df_open))

print(df_open_pilot.isna().sum())
df_open.head()

comment                     0
rating                      0
bigrams_afternostop_sent    0
ngrams_afternostop_sent     0
y                           0
dtype: int64


,comment,rating,bigrams_afternostop_sent,ngrams_afternostop_sent,y
0,tttsfddfhfdhdhdhdhf,1,tttsfddfhfdhdhdhdhf,tttsfddfhfdhdhdhdhf,0
1,This product is awesome! Super helpful with the nerve damage sustained from a dog bit incident. Couldnt recommended a better product,1,product_awesome super_helpful nerve_damage sustained dog_bit incident couldnt recommended better_product,product_awesome super_helpful nerve_damage sustained dog_bit incident couldnt recommended better_product,0
2,Super safe helmet and very comfortable,1,super safe helmet_comfortable,super safe helmet_comfortable,0
3,"I am pretty frustrated with him! My boyfriend decided to buy me one of the buffalo bags! He ordered it back in October and the at the time the website stated 3-4 to weeks and it would be sent out! He also accepted the money! We have tried and tried to talk to him and he ignores messages, he messaged back once! 11 weeks later and he is still ignoring us! Very very disappointed! I would love to support his work as it a gorgeous but he is sure it making it hard! He could at least message back... saying anything, possibly a hey I am super busy and I would be happy and understanding. Very disappointed!!!",1,pretty frustrated boyfriend decided_buy one buffalo bag_ordered back_october time website_stated 3_4 week_sent also accepted_money tried_tried talk ignores message messaged_back 11_week later_still ignoring u disappointed support work gorgeous sure making_hard could_least message_back saying_anything possibly hey super_busy happy understanding disappointed,pretty frustrated boyfriend decided_buy one buffalo bag_ordered back_october time website_stated 3_4 week_sent also accepted_money tried_tried talk ignores message messaged_back 11_week later_still ignoring u_disappointed support_work gorgeous sure making_hard could_least message_back saying_anything possibly hey super_busy happy understanding disappointed,0
4,"Stealing my money!!! Ordered a coat and returned it. Still waiting for refund 45 days later. They have n o excuse except to blame it on my bank...which is not correct, as I work in the POS industry. As of now they have stolen from me, and they do not care at all. Horrible!",1,stealing_money ordered_coat returned_still waiting_refund 45_day later n excuse except blame bank correct work po industry stolen care horrible,stealing_money ordered_coat returned_still waiting_refund 45_day later n excuse except blame bank correct work po industry stolen care horrible,0


In [ ]:
df_open.columns

Index(['comment', 'rating', 'bigrams_afternostop_sent',
       'bigrams_beforenostop_sent', 'ngrams_afternostop_sent',
       'ngrams_beforenostop_sent', 'y'],
      dtype='object')

In [70]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split

df = df_open.copy()
# number_of_row = 1000
# # number_of_row = len(df['y'] == 1])
# df = df[df['y'] == 1].sample(number_of_row, random_state= 123).append(df[df['y'] == 0].sample(number_of_row, random_state= 123))
# print(f'reduced df len: {len(df)}')
# # 'comment', 'rating', 'bigrams_afternostop_sent',
# #        'bigrams_beforenostop_sent', 'ngrams_afternostop_sent',
# #        'ngrams_beforenostop_sent', 'y'

# choosen_column_2vec = 'ngrams_afternostop_sent'

# vectorizer = CountVectorizer(ngram_range=(1, 1), max_features=1000)
# # vectorizer = TfidfVectorizer(ngram_range=(1, 3), max_features=1000)

# model_filename = f"{models_root_dir}our_bow_dict_model.pkl"
# with open(model_filename, 'rb') as file:
#     vectorizer = pickle.load(file)

X = vectorizer.transform(df['ngrams_afternostop_sent'].to_list())
# # feature_names = vectorizer.get_feature_names()
# # print(feature_names)

X = X.toarray()
X = np.array(X)
y = np.array(df['y'].to_list())

x_train, x_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=42)


In [ ]:
all_ngrams = []
for ngram_list in df.ngrams_afternostop_sent.to_list():
  all_ngrams.extend(ngram_list.split())

from collections import Counter
counter = Counter(all_ngrams)
counter.most_common(100)

In [ ]:
feature_names = vectorizer.get_feature_names()
feature_names[:100]

In [ ]:
from tqdm import tqdm
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import accuracy_score, recall_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import pickle

#################################################################################à
tree_model = tree.DecisionTreeClassifier(max_leaf_nodes=10, max_depth=5)
#################################################################################à
tree_model.fit(x_train, y_train)

# Save to file in the current working directory
model_filename = f"{models_root_dir}our_bow_dict_model.pkl"
with open(model_filename, 'wb') as file:
    pickle.dump(tree_model, file)
# Load from file
with open(model_filename, 'rb') as file:
    tree_model = pickle.load(file)

predicted = tree_model.predict(x_test)
print(f"""tree_model: 
{classification_report(y_test, predicted)}""")

#################################################################################à
logit_model = LogisticRegression(class_weight=None)
#################################################################################à
logit_model.fit(x_train, y_train)
# Save to file in the current working directory
model_filename = f"{models_root_dir}basic_logit_model.pkl"
with open(model_filename, 'wb') as file:
    pickle.dump(logit_model, file)
# Load from file
with open(model_filename, 'rb') as file:
    logit_model = pickle.load(file)

predicted = logit_model.predict(x_test)
print(f"""logit_model: 
{classification_report(y_test, predicted)}""")

#################################################################################à
random_model = RandomForestClassifier()
#################################################################################à
random_model.fit(x_train, y_train)
# Save to file in the current working directory
model_filename = f"{models_root_dir}basic_random_model.pkl"
with open(model_filename, 'wb') as file:
    pickle.dump(random_model, file)
# Load from file
with open(model_filename, 'rb') as file:
    random_model = pickle.load(file)

predicted = random_model.predict(x_test)
print(f"""random_model: 
{classification_report(y_test, predicted)}""")


tree_model:               precision    recall  f1-score   support

           0       0.86      0.08      0.14     33583
           1       0.52      0.99      0.68     33562

    accuracy                           0.53     67145
   macro avg       0.69      0.53      0.41     67145
weighted avg       0.69      0.53      0.41     67145

logit_model:               precision    recall  f1-score   support

           0       0.73      0.49      0.59     33583
           1       0.62      0.82      0.70     33562

    accuracy                           0.66     67145
   macro avg       0.67      0.66      0.65     67145
weighted avg       0.67      0.66      0.65     67145

